# Синтаксический анализ

Парсинг (синтаксический анализ) представляет собой процесс сопоставления последовательности слов или символов — так называемой формальной грамматике.

Посмотрим, как в [формате БНФ](https://en.wikipedia.org/wiki/Backus%E2%80%93Naur_form) описывается в python оператор ветвления. https://docs.python.org/3/reference/compound_stmts.html#if 
]


Формальная грамматика состоит из алфавита (набор символов) и правил порождения. То есть определение, "новое понятие - это вот такая композиция из уже введенных понятий".

При разборе текст разбивается на единицы (токены). У каждого токена есть определение что это такое. Терминальный токен - он состоит из символов алфавита.

Для синтаксического разбора можно использовать пакет [pyparsing](https://pythonhosted.org/pyparsing/pyparsing-module.html). Покажем как им пользоваться на примерах.

Документация: https://pyparsing-docs.readthedocs.io/en/latest/


In [1]:
# считаем, что этот импорт уже выполнен во всех примерах
import pyparsing as pp

## Пример 1. Разбор строки import

Дан текст вида
```python
import matplotlib.pyplot as plt
```
Хотим привести его в вид
```python
{ 'import': [ 'matplotlib', 'pyplot' ], 'as': 'plt' }
```

In [2]:
module_name = pp.Word(pp.alphas + '_')                              # имя модуля - это латинские буквы и _
full_module_name = module_name + pp.ZeroOrMore('.' + module_name)   # модуль и может быть через . еще имена модулей
import_as = pp.Optional('as' + module_name)                         # может быть, а может и не быть часть as алиас
parse_module = 'import' + full_module_name + import_as              # соберем все в единую строку, начинающуюся с import

In [3]:
# разберем с помощью этих правил строку
text = 'import        matplotlib.pyplot     as      plt'
parse_module.parseString(text)

ParseResults(['import', 'matplotlib', '.', 'pyplot', 'as', 'plt'], {})

In [4]:
# или в виде списка
parse_module.parseString(text).asList()

['import', 'matplotlib', '.', 'pyplot', 'as', 'plt']

## Suppress - уберем найденные токены из результирующего списка

То есть эти токены обязаны быть, но в результат они не войдут. Варианты использования (пример для разделяющей запятой)
```python
comma = Suppress( Literal(",") ) 
comma = Literal(",").suppress() 
comma = Suppress(",")
```

In [5]:
module_name = pp.Word(pp.alphas + '_')                              # имя модуля - это латинские буквы и _
full_module_name = module_name + pp.ZeroOrMore(pp.Suppress('.') + module_name)   # модуль и может быть через . еще имена модулей
import_as = pp.Optional(pp.Suppress('as') + module_name)                         # может быть, а может и не быть часть as алиас
parse_module = pp.Suppress('import') + full_module_name + import_as              # соберем все в единую строку, начинающуюся с import

text = 'import matplotlib.pyplot as plt'
parse_module.parseString(text).asList()

['matplotlib', 'pyplot', 'plt']

### Дадим имя правилу и будем обращаться по имени

* Все выражение справа заключим в `( )`
* После него поставим `('имя')`
* Можно обращаться по имени `.имя` к части разобранного выражения

In [2]:
import pyparsing as pp

In [3]:
module_name = pp.Word(pp.alphas + '_')
full_module_name = (module_name + pp.ZeroOrMore(pp.Suppress('.') + module_name)) ('modules')  # дали имя modules
import_as = (pp.Optional(pp.Suppress('as') + module_name))('import_as')                       # дали имя import_as
parse_module = pp.Suppress('import') + full_module_name + import_as



In [4]:

text = 'import matplotlib.pyplot as plt'
# text = 'import matplotlib.pyplot'
res = parse_module.parseString(text)
print(f'{res=}')
print(res.modules.asList())
#print('<'+res.import_as+'>', type(res.import_as))
print(res.import_as.asList())

res=ParseResults(['matplotlib', 'pyplot', 'plt'], {'modules': ['matplotlib', 'pyplot'], 'import_as': ['plt']})
['matplotlib', 'pyplot']
['plt']


## setParseAction - преобразуем в нужный формат через lambda

In [7]:
parse_module = (pp.Suppress('import') + full_module_name).setParseAction(lambda t: {'import': t.modules.asList(), 'as': t.import_as.asList()[0]})

In [2]:
import pyparsing as pp
text = 'import matplotlib.pyplot as plt'
module_name = pp.Word(pp.alphas + '_')
full_module_name = (module_name + pp.ZeroOrMore(pp.Suppress('.') + module_name)) ('modules')  # дали имя modules
import_as = (pp.Optional(pp.Suppress('as') + module_name))('import_as')                       # дали имя import_as
parse_module = (pp.Suppress('import') + full_module_name + import_as).setParseAction(lambda t: {'import': t.modules.asList(), 'as': t.import_as.asList()[0]})
parse_module.parseString(text)

ParseResults([{'import': ['matplotlib', 'pyplot'], 'as': 'plt'}], {})

In [9]:
parse_module.parseString(text).asList()

[{'import': ['matplotlib', 'pyplot'], 'as': 'plt'}]

In [10]:
parse_module.parseString(text).asList()[0]

{'import': ['matplotlib', 'pyplot'], 'as': 'plt'}

### Количество символов

Для указания точного количества символов используют аргументы [Word](https://pythonhosted.org/pyparsing/pyparsing.Word-class.html)

`__init__(self, initChars, bodyChars=None, min=1, max=0, exact=0, asKeyword=False, excludeChars=None)`       ''')

In [6]:
# US postal code can be a 5-digit zip, plus optional 4-digit qualifier
zip = pp.Combine(pp.Word(pp.nums, exact=5) + pp.Optional('-' + pp.Word(pp.nums, exact=4)))
zip.runTests('''
   # traditional ZIP code
   12345
   
   # ZIP+4 form
   12101-0001
   
   # invalid ZIP
   98765-
   ''')



# traditional ZIP code
12345
['12345']

# ZIP+4 form
12101-0001
['12101-0001']

# invalid ZIP
98765-
98765-
     ^
ParseException: Expected end of text, found '-'  (at char 5), (line:1, col:6)
FAIL: Expected end of text, found '-'  (at char 5), (line:1, col:6)


(False,
 [('12345', ParseResults(['12345'], {})),
  ('12101-0001', ParseResults(['12101-0001'], {})),
  ('98765-', Expected end of text, found '-'  (at char 5), (line:1, col:6))])

## Пробельные символы, начало и конец

По умолчанию все пробельные символы, начало и конец строки игнорируются. 

Проблема: при `real = Word(nums) + '.' + Word(nums)` этой схеме соответствуют не только `3.14159`, но и с пробелами внутри, то есть `3. 14159`. Используйте **Combine**, чтобы указать, что пробелы внутри не допустимы и возвращать надо единым токеном.

In [18]:
real = pp.Word(pp.nums) + '.' + pp.Word(pp.nums)
res = real.parseString('3.1415')
print(res)
res = real.parseString('3.   12')
print(res)

['3', '.', '1415']
['3', '.', '12']


In [20]:
real = pp.Combine(pp.Word(pp.nums) + '.' + pp.Word(pp.nums))
res = real.parseString('3.1415')    # ['3.1415']
print(res)
res = real.parseString('3.   12')   # ошибка
print(res)

['3.1415']


ParseException: Expected W:(0-9), found ' '  (at char 2), (line:1, col:3)

Не путать **Combine** с **Group**. 

**Group** дает только группировку в подсписке в результирующем списке разбора и не обращает внимания на пробелы.

Use the Group class to enclose logical groups of tokens within a sublist. This will help organize your results into more hierarchical form (the default behavior is to return matching tokens as a flat list of matching input strings).

* `ParserElement.set_default_whitespace_chars` по умолчанию содержит `' \t\n'`. Для грамматики, где символ `\n` значимый, стоит указать `' \t'`. Можно изменить и тогда весь дальнейший парсинг будет работать с новым набором пробельных символов.

*  Вызовите [leaveWhitespace()](https://pythonhosted.org/pyparsing/pyparsing.Forward-class.html#leaveWhitespace) на отдельное выражение перед его разбором, чтобы выключить пропуск пробельных символов при разборе. Disables the skipping of whitespace before matching the characters in the ParserElement's defined pattern. This is normally only used internally by the pyparsing module, but may be needed in some whitespace-sensitive grammars.tables).

| Токен | Означает |
|----|----|
| [White()](https://pythonhosted.org/pyparsing/pyparsing.White-class.html) | пробельный символ (пробел, \n, \t, \r) |
| `AtLineStart` | точно в начале строки  |
| `AtLineEnd` | точно в конце строки |
| `LineStart` | Matches if current position is at the beginning of a line within the parse string |
| `LineEnd` | Matches if current position is at the end of a line within the parse string. |
| `StringStart` | Matches if current position is at the beginning of the parse string. |
| `StringEnd` | Matches if current position is at the end of the parse string.  |
| `WordStart` | Matches if the current position is at the beginning of a Word, and is not preceded by any character in a given set of wordChars (default=printables characters) |
| `WordEnd` | Matches if the current position is at the end of a Word, and is not followed by any character in a given set of wordChars (default=printables characters). |

In [8]:
(pp.LineStart() + pp.Word(pp.alphas)).parseString("ABC")    # passes
(pp.LineStart() + pp.Word(pp.alphas)).parseString("  ABC")  # passes
(pp.Combine(pp.LineStart() + pp.Word(pp.alphas))).parseString("ABC")    # passes
(pp.Combine(pp.LineStart() + pp.Word(pp.alphas))).parseString("  ABC")  # fails
pp.AtLineStart(pp.Word(pp.alphas)).parseString("ABC")     # passes
pp.AtLineStart(pp.Word(pp.alphas)).parseString("  ABC")     # fails

ParseException: Expected start of line, found 'ABC'  (at char 2), (line:1, col:3)

## Базовые элементы 

| Класс | Пример | Что значит |
|----|----|----|
| Literal | `Literal('import')` | точное соотвествие строки | 
| Keyword | `Keyword('##')`, `Keyword('import')` | похоже на Literal, но требует, чтобы после него был пробельный символ или символ пунктуации; нужен, чтобы imported не разобрать как import и еще остаток. |
| CaselessKeyword | `` | похоже на Keyword, но игнорирует регистр |
| Char | `Char('#')` | ровно один символ |
| Word  | `Word('#')`,  `Word(nums)` | последовательность из перечисленных символов |
| CharsNotIn  |  | похоже на Word, но берем только те символы, что не перечислены |
| Regex  |  | регулярное выражение |


In [23]:
w1 = pp.Word('abcd')
text = w1 + pp.ZeroOrMore(w1)
print(text.parseString('baaacdac abcd abc ccc'))
print(text.parseString('abcdqqqbdc  aaa'))

['baaacdac', 'abcd', 'abc', 'ccc']
['abcd']


In [6]:
w3 = pp.Literal('abcd')
text = w3 + pp.ZeroOrMore(w3)
print(text.parseString('abcd abcd abcd badc'))
print(text.parseString('abcdd ccc baaacdac'))


['abcd', 'abcd', 'abcd']
['abcd']


In [16]:
w2 = pp.Keyword('abcd')
text = w2 + pp.ZeroOrMore(w2) 
print(text.parseString('abcd abcd abcd badc'))
print(text.parseString('abcd abcdz ccc baaacdac'))
#print(text.parseString('abcdqqqbdc  aaa'))  # должно быть хотя бы одно ключевое слово, а нет ни одного

['abcd', 'abcd', 'abcd']
['abcd']


ParseException: Expected Keyword 'abcd', keyword was immediately followed by keyword character, found 'qqqbdc'  (at char 4), (line:1, col:5)

In [18]:
ident = pp.Word(pp.alphas+"_", pp.alphanums+"_")
pp.Word(pp.srange("[a-zA-Z_]"), pp.srange("[a-zA-Z0-9_]"))

ip_address = pp.Word(pp.nums) + ('.' + pp.Word(pp.nums)) * 3

In [22]:
print(ident.parseString('x__12'))
print(ident.parseString('_'))
print(ident.parseString('_x12'))
#print(ident.parseString('3x__12'))

['x__12']
['_']
['_x12']


ParseException: Expected W:(A-Z_a-z, 0-9A-Z_a-z), found '3x'  (at char 0), (line:1, col:1)

## restOfLine = rest of line

In [39]:
h1 = pp.Keyword('#') + pp.White() + (pp.restOfLine())('lesson_title')
res = h1.parseString("# Название урока")
res
# как оставить только ['Название урока'] в списке разобранных токенов?

ParseResults(['#', ' ', 'Название урока'], {'lesson_title': 'Название урока'})

## Количество токенов

Можно указать сколько раз может быть повторен тот или иной токен при разборе.

| метод | regexp style | что означает |
|----|----|----|
| `` | `expr*3` | `expr+expr+expr` |
| `Optional(expr)` `Opt(expr)` | `expr[0,1]` | выражение есть или нет, оно опционально. |
|  `expr + expr + Opt(expr)` | `expr[2, 3]` | - два или три таких выражения |
| `ZeroOrMore(expr)` | `expr[...]`, `expr[0, ...]`, '`expr * ...` | ноль или более раз  |
| `OneOrMore(expr)` | `expr[1, ...]` | один или более раз  |
| `` | `expr[n, ...]` | n или более раз  |
| `` | `expr[..., n]` | не более n раз |


## Unicode символы

Специальные наборы алфавитов. Но! Нам это вообще не нужно.


In [56]:
# функция для нормальной печати юникодных символов при разборе
def bprint(obj):
    print(obj.__repr__().decode('string_escape'))


# Задача

Разобрать h2 заголовок с разметкой типов заголовка, default = TEXT, добавить SKIP как возможность двух типов.

## Документация

* [Индекс](https://pythonhosted.org/pyparsing/identifier-index.html) фукнций и классов
* https://pyparsing-docs.readthedocs.io/en/latest/HowToUsePyparsing.html
* http://xgu.ru/wiki/pyparsing - маленькая страница, но на русском
* http://netsago.org/ru/docs/1/8/ - на русском, разбор химических формул (возможно, перевод)
* http://s.arboreus.com/2009/07/easy-parsing-in-python.html - на русском, пример входных данных х и у из таблицы, Group
* https://habr.com/ru/articles/239081/ - статья на Хабре - разбор примера на import
* https://habr.com/ru/articles/241670/ - продолжение статьи на Хабре, примеры с размерностями русскими и степенями.

# Best Practice

https://github.com/pyparsing/pyparsing/wiki/Best-Practices#use-parse-actions

## Вопросы для самопроверки

1. Чем отличается Optional от ZeroOrMore?
2. Какие названия методов являются устаревшими? Optional, optional, |
3. Чем отличается Group от Combine?